<a href="https://colab.research.google.com/github/Triwka/ml_homework/blob/main/kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [147]:
SEED = 777

In [86]:
import numpy as np
import pandas as pd
import string
import datetime
import re

from geopy.distance import geodesic
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, average_precision_score
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, classification_report, precision_score, recall_score
from sklearn.preprocessing import StandardScaler, Normalizer
import joblib


#импортируем библиотеки matplotlib и seaborn для построения графиков
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [9]:
train = pd.read_csv('https://drive.google.com/uc?id=1pVqiPMX7L7bqviBCE-13_uA6ZfBCzOgw', sep=';')
test = pd.read_csv('https://drive.google.com/uc?id=1YtjNEL0Yye6tkW26vpDbfQQv3dZvpSYr', sep=';')
submission = pd.read_csv('https://drive.google.com/uc?id=1MTZnySU5mGdppCp9zq8mg-Of1I_rSai8', sep=';')

Перевод interest_level в цифры


In [74]:
mapper={
        'low':0,
        'medium':1,
        'high':2
       }
train['interest_level'] = train['interest_level'].apply(lambda x: mapper[x])

In [10]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
0,1.0,0,cb0ecd5356125b546833239efdb154c2,2016-06-02 04:21:46,AMAZING STUDIO!! CONVENIENT TO ALL - G...,Gold Street,"['Cats Allowed', 'Dogs Allowed']",40.6969,7097709,-73.9830,79d1809d224bc7f26c719f247d6c8dc6,['https://photos.renthop.com/2/7097709_8f74305...,2575,257 Gold Street,low
1,1.0,2,4df46d9768a98df24200a2e7693370bd,2016-05-26 05:44:12,"Near NYU, Washington SQ Park, centrally locate...",Thompson Street,[],40.7264,7071860,-74.0013,e60b02e0cfd2eea7d777bb283d6f8d4d,['https://photos.renthop.com/2/7071860_32b196a...,3250,128 Thompson Street,high
2,1.0,0,0,2016-06-17 01:22:32,RENT STABILIZED! FULL TIME DOORMAN.LEXINGTON A...,East 35th Street,"['Doorman', 'Dogs Allowed', 'Cats Allowed']",40.7466,7174524,-73.9779,91a276fdb4c7ae5ff56fd0884b5121db,['https://photos.renthop.com/2/7174524_bde8df9...,2448,165 East 35th Street,low
3,1.0,1,78e50c94b33bb1ceeb5f4c8ff7455703,2016-06-03 05:26:40,WHY TO LIVE IN THIS LUXURIOUS BUILDING? WEEKDA...,West 42nd Street,"['Swimming Pool', 'Roof Deck', 'Dining Room', ...",40.7610,7104515,-73.9992,cbcd513212d592e6c717d27be14f46ac,['https://photos.renthop.com/2/7104515_bab0753...,3300,610 West 42nd Street,low
4,2.0,0,e87f1236b102798c1fa104a372800444,2016-04-12 04:10:28,Listed on the National Register of His...,Greenwich Street,"['Doorman', 'Fitness Center', 'No Fee']",40.7324,6859942,-74.0081,b209e2c4384a64cc307c26759ee0c651,['https://photos.renthop.com/2/6859942_16f88f7...,5450,666 Greenwich Street,low


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39481 entries, 0 to 39480
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bathrooms        39481 non-null  float64
 1   bedrooms         39481 non-null  int64  
 2   building_id      39481 non-null  object 
 3   created          39481 non-null  object 
 4   description      38337 non-null  object 
 5   display_address  39376 non-null  object 
 6   features         39481 non-null  object 
 7   latitude         39481 non-null  float64
 8   listing_id       39481 non-null  int64  
 9   longitude        39481 non-null  float64
 10  manager_id       39481 non-null  object 
 11  photos           39481 non-null  object 
 12  price            39481 non-null  int64  
 13  street_address   39472 non-null  object 
 14  interest_level   39481 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 4.5+ MB


In [13]:
train.duplicated().sum()

0

In [15]:
train.isna().sum()

bathrooms             0
bedrooms              0
building_id           0
created               0
description        1144
display_address     105
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        9
interest_level        0
dtype: int64

In [16]:
train.description = train.description.fillna('empty')
train = train.dropna()

## 1.Добавление новых фичей


1)Добавление фичей "количество атрибутов квартиры" и "количество фотографий"

In [17]:
train['num_features']=train['features'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
train['num_photos']=train['photos'].str.replace('[\[\]\']', '').str.split(', ').apply(len)

<ipython-input-17-6a11e9b84264>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['num_features']=train['features'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
<ipython-input-17-6a11e9b84264>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['num_photos']=train['photos'].str.replace('[\[\]\']', '').str.split(', ').apply(len)


2)ГИПОТЕЗА:популярость квартиры зависит от её удалённости до какого-либо объекта - добавление фичей "удалённость до какого-либо популярного объекта"

In [18]:
central_park = tuple([40.7736,-73.9727]) #Координаты центрального парка
time_square = tuple([40.7577,-73.9857])  #Координаты тайм-сквера
wall_street = tuple([40.7072,-74.0116])   #Координаты уолл-стрит

In [19]:
train['distance_central_park'] = train.latitude
train['distance_time_square'] = train.latitude
train['distance_wall_street'] = train.latitude

In [21]:
for i in range(train.shape[0]):
  train.distance_central_park.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),central_park).miles

for i in range(train.shape[0]):
  train.distance_time_square.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),time_square).miles

for i in range(train.shape[0]):
  train.distance_wall_street.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),wall_street).miles

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Также из лора гамбургероголовых(осуждаю) можно сказать выдвинуть гипотезы, что популярность квартиры зависит от наличия в шаговой доступности: метро, школы, церкви, удалённость от какого-нибудь гетто, но я не знаю как добавить информацию об этом в датафрейм. С метро можно было бы также с помощью geopy вычислить расстояние до каждой станции и добавлять меньшее для каждого объявления, но станций 450+, а списком их координаты я не нашёл.

3)Некоторые фичи из списка атрибутов квартиры


In [22]:
train['dogs_allowed'] = train.features.apply(lambda x: 1 if 'Dogs Allowed' in x else 0) #разрешены ли собаки
train['cats_allowed'] = train.features.apply(lambda x: 1 if 'Cats Allowed' in x else 0) #разрешены ли кошки
train['doorman'] = train.features.apply(lambda x: 1 if 'Doorman' in x else 0) #есть ли консьерж

In [23]:
train[train.interest_level == 'low'].price.describe()

count    2.737500e+04
mean     4.067016e+03
std      1.178325e+04
min      4.300000e+01
25%      2.625000e+03
50%      3.300000e+03
75%      4.400000e+03
max      1.150000e+06
Name: price, dtype: float64

In [24]:
train[train.interest_level == 'high'].price.describe()

count      3080.000000
mean       2692.612338
std        2250.406333
min         700.000000
25%        1850.000000
50%        2400.000000
75%        3104.000000
max      111111.000000
Name: price, dtype: float64

In [25]:
train[train.interest_level == 'medium'].price.describe()

count     8920.000000
mean      3158.442152
std       1247.192766
min        695.000000
25%       2300.000000
50%       2890.000000
75%       3650.000000
max      15000.000000
Name: price, dtype: float64

Из вышеприведенной статистики видно, что средняя цена "горячего" объявления - 2692
(у него и отклонение отличается аж в два раза от остальных, но пока я не знаю какую информацию можно из этого извлечь)

Вообще, надо бы уточнить что из себя представляет interest_level, потому что если это популярность квартиры(считается что человек её очно просмотрел) то цена имеет меньше влияние, а если это популярность объявления(количество кликов на объявление), то цена имеет фактор кликбейта(указать привлекательную-человек заходит, а там сверху надбавки за атрибуты разные коммуналка и проч.)
4)
Следующая фича сгенерирована на основе того, что популярность объявления высчитана путем подсчета количества кликов на объявление


In [26]:
train['price_lower_than_2700'] = train.price.apply(lambda x: 1 if x < 2700 else 0)

In [ ]:
train[5:]

5)ГИПОТЕЗА: информация о fee в объявлении также влияет на то кликнет на неё пользователь или нет

In [34]:
#функция для удаления символов пунктуации и цифр
def del_puncdigit(string):
    return ' '.join((re.sub(r'[№"\'_/.:?!\d()%<>;,+#$&\s+]', u' ', string)).lower().split())
               


In [40]:
train['fee_inf'] = train.description.apply(lambda x: 1 if 'fee' in del_puncdigit(x).split() else 0)

6)ГИПОТЕЗА: популярность объявления зависит от менеджера

In [48]:
c00l_managers = train[train.interest_level == 'high'].manager_id.to_list() #создаем список "крутых менеджеров", у которых "горячие" объявления, чтобы проверять наличие конкретного в этом списке

In [49]:
train['cool_manager'] = train.manager_id.apply(lambda x: 1 if x in c00l_managers else 0)

In [92]:
cat_columns = []
num_columns = []

for column_name in train.columns:
    if (train[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

Категориальные данные:	  ['building_id', 'description', 'display_address', 'features', 'manager_id', 'photos', 'street_address', 'clean_desc'] 
 Число столблцов =  8
Числовые данные:	  ['bathrooms', 'bedrooms', 'created', 'latitude', 'listing_id', 'longitude', 'price', 'interest_level', 'num_features', 'num_photos', 'distance_central_park', 'distance_time_square', 'distance_wall_street', 'dogs_allowed', 'cats_allowed', 'doorman', 'price_lower_than_2700', 'fee_inf', 'cool_manager', 'created_hour'] 
 Число столблцов =  20


In [52]:
train[num_columns]

,bathrooms,bedrooms,latitude,listing_id,longitude,price,num_features,num_photos,distance_central_park,distance_time_square,distance_wall_street,dogs_allowed,cats_allowed,doorman,price_lower_than_2700,fee_inf,cool_manager
0,1.0,0,40.6969,7097709,-73.9830,2575,2,10,5.320034,4.197741,1.661572,1,1,0,1,1,0
1,1.0,2,40.7264,7071860,-74.0013,3250,1,5,3.586089,2.309755,1.430961,0,0,0,0,0,1
2,1.0,0,40.7466,7174524,-73.9779,2448,3,1,1.882948,0.868433,3.243590,1,1,1,1,0,0
3,1.0,1,40.7610,7104515,-73.9992,3300,16,6,1.639738,0.744031,3.768959,0,0,1,0,1,1
4,2.0,0,40.7324,6859942,-74.0081,5450,3,8,3.395988,2.104669,1.748541,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39476,1.0,3,40.7446,7183085,-73.9740,3795,2,11,2.002244,1.092725,3.248987,1,1,0,0,0,0
39477,1.0,0,40.7085,6927008,-74.0165,2725,7,1,5.046088,3.760205,0.272484,0,0,1,0,0,1
39478,1.0,0,40.5956,6852162,-73.7555,999,2,3,16.763670,16.472881,15.506220,0,0,0,1,0,0
39479,1.0,1,40.7671,7103463,-73.9529,2450,7,4,1.131412,1.839080,5.155192,1,1,0,1,0,1


7)Из базы по рекламе знаю, что постить для большего охвата лучше во время обеда (12:00 - 13:00) либо после работы


In [71]:
train["created"] = pd.to_datetime(train["created"])
train["created_hour"] = train.created.dt.hour

Удалим сильно коррелирующие между собой признаки

In [116]:
dropped = ['latitude','distance_time_square','distance_wall_street','dogs_allowed']

In [112]:
train.drop(dropped, axis = 1)[].columns

Index(['bathrooms', 'bedrooms', 'building_id', 'description',
       'display_address', 'features', 'listing_id', 'longitude', 'manager_id',
       'photos', 'price', 'street_address', 'num_features', 'num_photos',
       'distance_central_park', 'cats_allowed', 'doorman',
       'price_lower_than_2700', 'clean_desc', 'fee_inf', 'cool_manager',
       'created_hour'],
      dtype='object')

## 2. Тестирование различных алгоритмов

In [133]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, classification_report, precision_score, recall_score
import joblib

In [99]:
num_columns = ['bathrooms',
 'bedrooms',
 'latitude',
 'listing_id',
 'longitude',
 'price',
 'num_features',
 'num_photos',
 'distance_central_park',
 'distance_time_square',
 'distance_wall_street',
 'dogs_allowed',
 'cats_allowed',
 'doorman',
 'price_lower_than_2700',
 'fee_inf',
 'cool_manager',
 'created_hour']

In [76]:
features = ['bathrooms', 'bedrooms', 'listing_id','longitude','price','num_features','num_photos','cats_allowed','doorman','price_lower_than_2700','fee_inf','cool_manager','created_hour']
target = 'interest_level'

In [125]:
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train[num_columns])

In [126]:
X_train, X_test, y_train, y_test = train_test_split(scaled_train, train[target],
                                                    test_size=0.25, random_state=777)

In [129]:
lg = LogisticRegression(random_state = 777)
lg.fit(X_train, y_train)
predicts = lg.predict(X_test)
predicts_proba = lg.predict_proba(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [130]:
f1_score(y_test, predicts, average='macro')

0.3902501663031602

In [ ]:
--------------------------------------

In [134]:
scv = LinearSVC(random_state = 777)
scv.fit(X_train, y_train)
predicts = scv.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [135]:
f1_score(y_test, predicts, average='macro')

0.3226419224096945

In [136]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
predicts = knn.predict(X_test)

In [137]:
f1_score(y_test, predicts, average='macro')

0.4253689289104075

In [138]:
dt = DecisionTreeClassifier(random_state=777, class_weight='balanced')
dt.fit(X_train, y_train)
predicts = dt.predict(X_test)

In [139]:
f1_score(y_test, predicts, average='macro')

0.4939893421849724

In [140]:
rf = RandomForestClassifier(n_estimators = 100, random_state=777)
rf.fit(X_train, y_train)
predicts = rf.predict(X_test)

In [141]:
f1_score(y_test, predicts, average='macro')

0.5271793703291646

In [142]:
xgb = XGBClassifier(seed=777)
xgb.fit(X_train, y_train)
predicts = xgb.predict(X_test)

In [143]:
f1_score(y_test, predicts, average='macro')

0.4696651191210856

In [144]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
params = {
    'max_depth': [3, 4, None, 5],
    "min_samples_leaf": list(range(1, 5))
}

rf = RandomForestClassifier(n_estimators=100, random_state=42, 
                             n_jobs=-1, oob_score=True)

dt_random_search = RandomizedSearchCV(rf, params, n_jobs=-1, cv=skf, verbose=1)
dt_random_search.fit(X_train, y_train)

print(dt_random_search.best_estimator_)
print(dt_random_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
RandomForestClassifier(min_samples_leaf=2, n_jobs=-1, oob_score=True,
                       random_state=42)
{'min_samples_leaf': 2, 'max_depth': None}


In [145]:
predicts = dt_random_search.predict(X_test)

In [146]:
f1_score(y_test, predicts, average='macro')

0.5192362869258198

In [ ]:
rf = dt_random_search.best_estimator_
rf.fit(train[features], train[target])